In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Läs in Excel-filer
Solprod = pd.read_excel("SolprodABK.xlsx")
ABKEl = pd.read_excel("ABKEL.xlsx", names=["ABKEl"])
Spotpris = pd.read_excel("Spotpris.xlsx", names=["Spotpris"])  # Namnge kolumnen korrekt

# Extrahera första kolumnen från respektive DataFrame
Sol = 0.8*Solprod.iloc[:, 0]  # Solel-produktion

# Konvertera Spotpris till en lista för iteration
Spot_list = Spotpris["Spotpris"].tolist()
ABK_list = ABKEl["ABKEl"].tolist()

# Skapa tidsindex för ABKEl och Spotpris
ABKEl["Tid"] = pd.date_range(start="2023-01-01 00:00", periods=len(ABKEl), freq="H")
ABKEl.set_index("Tid", inplace=True)

Spotpris["Tid"] = pd.date_range(start="2023-01-01 00:00", periods=len(Spotpris), freq="H")
Spotpris.set_index("Tid", inplace=True)

# Resampla till månatligt genomsnitt
monthly_avg_ABK = ABKEl["ABKEl"].resample("M").mean()
monthly_avg_E = Spotpris["Spotpris"].resample("M").mean()

# Beräkna överföringsavgift per månad
Spotpris["E"] = 4.004 + 0.0613 * Spotpris["Spotpris"]
monthly_avg_E = Spotpris["E"].resample("M").mean()

# Beräkna summa överföringsavgift för hela året
summaÖverfAvgift = (monthly_avg_E * monthly_avg_ABK).sum()
fastAbbKostnad = 795180.4
summaElnätskostnadKR = (summaÖverfAvgift / 100) + fastAbbKostnad

Spot_array = np.array(Spot_list)
ABK_array = np.array(ABK_list)

elhandelskostnad = (Spot_array * ABK_array).sum() / 100
energiskatt = (43.9 * ABK_array).sum() / 100

summaElKostnad = (summaElnätskostnadKR + elhandelskostnad + energiskatt) * 1.25

print("Summa kostnad i el 2023:", summaElKostnad)

# Ta fram elförbrukning (första kolumnen) som en Pandas Series
elförbrukning = pd.Series(ABKEl.iloc[:, 0])

# Sortera elförbrukningen i fallande ordning och hämta de 30 högsta värdena
top_30_hours = elförbrukning.sort_values(ascending=False).head(30)

# Skapa en DataFrame med radnummer (index) och värde (elförbrukning)
top_30_with_index = top_30_hours.reset_index()

# Byt namn på kolumnerna för bättre läsbarhet
top_30_with_index.columns = ['Timme', 'Elförbrukning']

# Skriv ut de 30 timmarna med högst elförbrukning och radnummer
print("Original elförbrukning topptimmar: " + top_30_with_index.to_string())



# ----------------------- YEAR 1 SIMULATION -----------------------
# Initialize Battery and Costs
batteriKapacitet = 0.0  # Start with an empty battery
batteriMaxKapacitet = 5000  # Max battery capacity in kWh
maxFrånBatteri = 1500

#Variabler som används i if-satser för kontroll av extremvärden av effekttopparna
#maxförbrukning_för_lagring = 1480

hög_efterfrågan_gräns = 1480

medelhög_efterfrågan_gräns = 1400

# Lists to store values for plotting
elförbrukning = []  # Adjusted electricity consumption for Year 1
battery_values = []  # Battery energy over time
original_consumption = ABKEl["ABKEl"].tolist()
köptaTimmar = []  # Lista för köpta spotpriser och tid


for tid, sol, abk, spot in zip(Spotpris.index, Sol, ABK_list, Spot_list):
    diff = abk - sol  # Difference between consumption and solar production


    if diff < 0:  
        # Om solproduktionen är större än förbrukningen, ladda batteriet istället för att spara negativa värden
        batteriKapacitet = min(batteriKapacitet - diff, batteriMaxKapacitet)  # -diff eftersom diff är negativ
        elförbrukning.append(0)  # Vi behöver inte köpa el
        köptaTimmar.append((tid,0))
    else:    
        # Dynamisk justering av kapningsfaktor beroende på efterfrågan
        if diff > hög_efterfrågan_gräns:  # Om efterfrågan är extremt hög
            faktor = 0.35  # Aggressiv kapning vid extrema toppar
        elif diff > medelhög_efterfrågan_gräns:  # Om efterfrågan är måttligt hög
            faktor = 0.15  # Medelhög kapning
        else:  # Om efterfrågan är låg
            faktor = 0.1  # Mjukare kapning vid normal efterfrågan

        if diff >= hög_efterfrågan_gräns:
            energiFrånBatteri = min(batteriKapacitet * faktor, diff * faktor, maxFrånBatteri)
            batteriKapacitet -= energiFrånBatteri
            köpt_el = max(0, abk - energiFrånBatteri)
            elförbrukning.append(köpt_el)  
            köptaTimmar.append((tid, spot if köpt_el > 0 else 0))
        else:
            if spot <= 35 and sol > 0:
                if batteriKapacitet + sol >= 5000: 
                    solÖver = (batteriKapacitet + sol) - 5000
                    batteriKapacitet = min(batteriKapacitet + sol, batteriMaxKapacitet)
                    elförbrukning.append(max(0, abk-solÖver))  
                    köptaTimmar.append((tid,spot))
                else:
                    batteriKapacitet = min(batteriKapacitet + sol, batteriMaxKapacitet)
                    elförbrukning.append(abk)  
                    köptaTimmar.append((tid, spot))
                                
            elif 35 < spot < 100 and sol > 0:
                elförbrukning.append(diff)
                köptaTimmar.append((tid, spot))
            elif spot >= 100:
                energiFrånBatteri = min(batteriKapacitet * faktor, diff * faktor, maxFrånBatteri)
                batteriKapacitet -= energiFrånBatteri
                köpt_el = max(0, diff - energiFrånBatteri)
                elförbrukning.append(köpt_el)  
                köptaTimmar.append((tid, spot if köpt_el > 0 else 0))
                      
            else:
                elförbrukning.append(abk)  
                köptaTimmar.append((tid, spot))

    '''
    # 📌 Lägg till denna kontroll efter att batterikapaciteten uppdaterats
    if timme in [2152, 2128, 2129, 8104, 2153, 8080, 2104, 2105, 7984, 8032]:  
        print(f"Timme {timme}: Batterikapacitet = {batteriKapacitet}, Diff = {diff}, Spotpris = {spot}")
        '''
       
    battery_values.append(batteriKapacitet)
    

# Konvertera elförbrukning till en NumPy-array
elförbrukning_array = np.array(elförbrukning)

# Hitta de 30 högsta värdena i elförbrukning
top_30_indices = np.argsort(elförbrukning_array)[-30:]  # Index för de 30 största värdena

# Hämta de faktiska tidsstämplarna från Spotpris.index
top_30_times = Spotpris.index[top_30_indices]

# Hämta de 30 största elförbrukningsvärdena
top_30_values = elförbrukning_array[top_30_indices]

# Sortera topp 30 i fallande ordning och behåll index
sorted_pairs = sorted(zip(top_30_values, top_30_times), reverse=True)

# Skriv ut de 30 högsta värdena tillsammans med originaltimmarna
print("De 30 högsta elförbrukningsvärdena och deras timmar i Excel-filen:")

for i, (value, original_time) in enumerate(sorted_pairs):
    excel_rad = i + 8  # +8 för rubrikrad och 1-baserade index, och +1 för att börja från 1
    print(f"{i+1}. Timme {original_time}  - Förbrukning: {value} (Excel rad: {excel_rad})")



'''
# Skapa tomma listor för att lagra timmar och diff-värden
diff_lista = []

# Din stora for-loop där du beräknar diff och andra parametrar
for timme, (sol, abk, spot) in enumerate(zip(Sol, ABK, Spot)):
    diff = abk - sol  # Skillnaden mellan efterfrågan och solproduktion
    
    timmar.append(timme)  # Lägg till timme
    diff_lista.append(diff)  # Lägg till diff-värdet
    

    # Här kan du ha dina if-satser och logik som tidigare
    # (För att hantera batteriladdning, elköp etc.)

# Nu när for-loopen har körts klart, skapa plottningen en gång.
plt.figure(figsize=(10, 6))
plt.plot(timmar, diff_lista, label="Diff (ABK - Solproduktion)", color='blue')
plt.xlabel("Tid (timmar)")
plt.ylabel("Diff (kW)")
plt.title("Diff mellan efterfrågan och solproduktion över året")
plt.grid(True)
plt.legend()

# Justera y-axelns intervall
min_diff = min(diff_lista)
max_diff = max(diff_lista)
tick_interval = (max_diff - min_diff) / 50  # Dela upp i 10 sektioner (justera om behövs)

plt.yticks(np.arange(min_diff, max_diff, tick_interval))  # Sätt tickvärden

plt.show()
'''

# Fyll på elförbrukningen om den är kortare än Spotpris.index
if len(elförbrukning) < len(Spotpris.index):
    elförbrukning.extend([0] * (len(Spotpris.index) - len(elförbrukning)))


# Skapa DataFrame med tid som index
köptaTimmar_df = pd.DataFrame(köptaTimmar, columns=["Tid", "Spotpris"])
köptaTimmar_df.set_index("Tid", inplace=True)

# Beräkna månatligt medelvärde för köpta spotpriser
monthly_avg_E_köpt = köptaTimmar_df["Spotpris"].resample("M").mean()

# Skapa DataFrame för den uppdaterade elförbrukningen
elförbrukning_df = pd.DataFrame({"Tid": Spotpris.index, "elförbrukning": elförbrukning})
elförbrukning_df.set_index("Tid", inplace=True)

# Beräkna månatligt medelvärde av den uppdaterade elförbrukningen
monthly_avg_elförbrukning = elförbrukning_df["elförbrukning"].resample("M").mean()

# Beräkna justerad överföringsavgift
summaJusteradÖverf = (monthly_avg_E_köpt * monthly_avg_elförbrukning).sum()

summaJusteradÖverfKR = (summaJusteradÖverf / 100) + fastAbbKostnad

# Filtrera elförbrukning endast för de köpta timmarna
elförbrukning_köpt = elförbrukning_df.loc[köptaTimmar_df.index]

# Beräkna elhandelskostnaden: Spotpris * elförbrukning
elhandelskostnad_köpt = (köptaTimmar_df["Spotpris"] * elförbrukning_köpt["elförbrukning"]).sum() / 100

# Beräkna den justerade energiskatten
total_elförbrukning = sum(elförbrukning)
JusteradEnergiskatt = (total_elförbrukning * 43.9) / 100

# Beräkna den justerade elkostnaden
summaJusteradElkostnad = 1.25 * (JusteradEnergiskatt + elhandelskostnad_köpt + summaJusteradÖverfKR)

print("Justerad elkostnad simulering:", summaJusteradElkostnad)

# ----------------------- PLOTTING RESULTS -----------------------
fig, ax = plt.subplots(3, 1, figsize=(12, 10))

ax[0].plot(battery_values, label='Batteri år 1', color='blue')
ax[1].plot(elförbrukning, label='Justerad elförbrukning år 1', color='red')
ax[2].plot(original_consumption, label='Original elförbrukning', color='green')

for a in ax:
    a.legend()
    a.grid(True)

plt.tight_layout()
plt.show()

'''
# ----------------------- YEAR 2 SIMULATION -----------------------
batteriKapacitet = battery_energy_end_of_year1  # Start Year 2 with leftover battery energy
KöptElSpotpris = 0.0  # Reset for Year 2
elförbrukning_year2 = []
battery_values_year2 = []

for sol, abk, spot in zip(Sol, ABK, Spot):
    diff = abk - sol

    if spot <= 35 and sol > 0:
        batteriKapacitet = min(batteriKapacitet + sol, batteriMaxKapacitet)
        KöptElSpotpris += spot * abk
        elförbrukning_year2.append(abk)
    elif 35 < spot < 100 and sol > 0 and diff > 0:
        KöptElSpotpris += spot * diff
        elförbrukning_year2.append(diff)
    elif spot >= 100 and sol > 0 and diff > 0 and batteriKapacitet > 0:
        energiFrånBatteri = min(batteriKapacitet, diff, maxFrånBatteri)
        batteriKapacitet -= energiFrånBatteri
        KöptElSpotpris += (diff - energiFrånBatteri) * spot
        elförbrukning_year2.append(diff - energiFrånBatteri)
    elif spot > 35 and sol > 0 and diff < 0:
        batteriKapacitet = min(batteriKapacitet + (-diff), batteriMaxKapacitet)
    elif sol == 0 and batteriKapacitet > 0 and spot >= 100:
        energiFrånBatteri = min(batteriKapacitet, abk, maxFrånBatteri)
        batteriKapacitet -= energiFrånBatteri
        KöptElSpotpris += (abk - energiFrånBatteri) * spot
        elförbrukning_year2.append(max(0, abk - energiFrånBatteri))
    else:
        KöptElSpotpris += abk * spot
        elförbrukning_year2.append(abk)

    battery_values_year2.append(batteriKapacitet)

köptEliKr_år2 = KöptElSpotpris / 100
besparingar_år2 = SummaSpotpris2023 - köptEliKr_år2

print("\n--- År 2 Slut ---")
print("Batterienergi kvar vid årets slut:", batteriKapacitet, "kWh")
print("Summa spotpris köpt el med batteri år 2:", köptEliKr_år2)
print("Besparingar i spotpris år 2:", besparingar_år2)

# ----------------------- PLOTTING RESULTS -----------------------
fig, ax = plt.subplots(5, 1, figsize=(12, 14))

ax[0].plot(battery_values, label='Batteri år 1', color='blue')
ax[1].plot(battery_values_year2, label='Batteri år 2', color='cyan', linestyle='dashed')
ax[2].plot(elförbrukning, label='Justerad elförbrukning år 1', color='red')
ax[3].plot(elförbrukning_year2, label='Justerad elförbrukning år 2', color='orange', linestyle='dashed')
ax[4].plot(original_consumption, label='Original elförbrukning', color='green')

for a in ax:
    a.legend()
    a.grid(True)

plt.tight_layout()
plt.show()
'''
